In [97]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [98]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [99]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [100]:
news_dataset = pd.read_csv('/content/train.csv')

In [101]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [102]:
news_dataset.shape

(20800, 5)

In [103]:
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [104]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [105]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [106]:
news_dataset.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0
content,2515


In [107]:
news_dataset['label'].value_counts()

,count
label,
1,10413
0,10387


In [108]:
news_dataset = news_dataset.fillna('')

In [109]:
ps = PorterStemmer()

In [110]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [111]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [112]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [113]:
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [114]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [115]:
print(Y)

[1 0 1 ... 0 1 1]


In [116]:
Y.shape

(20800,)

In [117]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [118]:
print(X)

  (0, 14626)	0.28553358349485225
  (0, 12567)	0.2563589820798857
  (0, 8310)	0.3596536533885757
  (0, 8048)	0.2934713957861362
  (0, 7190)	0.24644399370485162
  (0, 6552)	0.21885788468666617
  (0, 4637)	0.23132772852542255
  (0, 3543)	0.2689082732577136
  (0, 3359)	0.3596536533885757
  (0, 2757)	0.24749629983427848
  (0, 2312)	0.37305738773975167
  (0, 247)	0.2702588721108487
  (1, 15663)	0.30543189988638736
  (1, 6377)	0.19530026425832656
  (1, 5140)	0.7101106653194483
  (1, 3328)	0.2633913741576576
  (1, 2619)	0.19610930415709218
  (1, 2066)	0.38047138877486064
  (1, 1764)	0.15430291936631466
  (1, 1391)	0.29649664509588236
  (2, 14560)	0.41792081528179686
  (2, 8973)	0.4931379270922205
  (2, 5579)	0.35039256210774505
  (2, 5031)	0.3876367834516749
  (2, 2895)	0.4571578367807032
  :	:
  (20797, 12239)	0.25299971331630106
  (20797, 11515)	0.2746816397264756
  (20797, 11321)	0.2462639038918503
  (20797, 9605)	0.08009780291419748
  (20797, 8942)	0.17302467550398123
  (20797, 8879)	0.292

In [119]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2, stratify=Y)

In [120]:
print(X.shape,X_train.shape,X_test.shape)

(20800, 15960) (16640, 15960) (4160, 15960)


In [121]:
model = LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

In [122]:
training_data_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(training_data_prediction,Y_train)
print('Accuracy score of training data',training_data_accuracy*100)

Accuracy score of training data 98.64182692307692


In [123]:
test_data_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(test_data_prediction,Y_test)
print('Accuracy score of test data',test_data_accuracy*100)

Accuracy score of test data 98.02884615384615


In [137]:
i = 29

In [138]:
input_data = (X_test[i])
prediction = model.predict(input_data)
print(prediction)
print(Y_test[i])
if(prediction[0] == 0):
  print('The news is Real')
else:
  print('The news is Fake')



[0]
0
The news is Real
